In [1]:
from src.Wine.Utils import create_directory,read_yaml,download_data_from_s3
from src.Wine.loggers import logger
from src.Wine.Exception import CustomException
from src.Wine.Constants import *
import os,sys
from pathlib import Path
from dataclasses import dataclass


In [2]:
#step3)update the entity file --->is nothing we r defining the class variable
#which was used in yaml file and futhure taking rtn as function

@dataclass
class DataValidationConfig():
    #defining class variable along with dtypes
    root_dir_path:Path
    unzip_data_dir_path: Path
    status_file: Path
    all_schema:dict

In [3]:
#step4)update the configuration Manager file
#in this configuration manager file we r reading the yaml file and creating directories
#and assgining value to class variable and taking returen as function

class ConfigurationManager():
    #creating constructor to initialize the instance variable
    def __init__(self,config_filepath = CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH,schema_filepath=SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath) #rtn as configbox dictatonary
        self.param =  read_yaml(param_filepath) #rtn as configbox dictionary
        self.schema = read_yaml(schema_filepath) #rtn as configbox dictionary

        #creating artifacts directory in the project structure
        create_directory([self.config.artifacts_root]) #it will create artifacts directory

    def get_data_validation(self) ->DataValidationConfig:
        #initializing the local variable
        config = self.config.data_validation
        schema = self.schema

        #creating directory data_validation
        create_directory([config.root_dir_path])

        #creating an object of DataValidationConfig class and assigning value to class variable
        data_validation_config = DataValidationConfig(
            root_dir_path=config.root_dir_path,
            unzip_data_dir_path=config.unzip_data_dir_path,
            status_file=config.status_file,
            all_schema=schema.Columns
        )
        return data_validation_config


In [4]:
import pandas as pd

In [5]:
#step5)Updating the Component File here we r doing or initiating the data validation Process!!!
#and also assigning the class variable as an object of (instance variable)
class DataValidation():
    def __init__(self,config:DataValidationConfig): #this config hold the value of DataValidationConfig class ki
        self.config = config

    def validating_all_column(self) ->bool:
        try:
            validation_status= None
            logger.info(f"Reading the Csv file {self.config.unzip_data_dir_path}")
            df = pd.read_csv(self.config.unzip_data_dir_path)   

            #getting all the column from csv and storing them into lst object
            lst_column = df.columns.to_list()

            #fetching the column name from schema.yaml se
            all_schema = self.config.all_schema.keys()

            #now iterating column from lst_column and checking them it exist in all_schema or not
            #or we can also validate the dtype of column also
            for col in lst_column:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.status_file,"w") as f:
                        f.write(f"Validation status: {validation_status}")

                else:
                    validation_status=True
                    with open(self.config.status_file,"w") as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status
        
        except Exception as e:
            raise CustomException(e,sys)

In [6]:
os.chdir('../')
%pwd

'd:\\Wine_ML_AlGO\\WineQualityModel'

In [7]:
#step6:- training the pipeline
try:
    #creating an object of configuration manager class
    cm = ConfigurationManager()
    data_validation_config = cm.get_data_validation()

    #creating an oject of Datavalidation class
    dv = DataValidation(data_validation_config)

    result = dv.validating_all_column()
    print(result)
    
except Exception as e:
    raise CustomException(e,sys)

[2024-09-26 18:19:16,051]-33-Reading yaml file config\config.yaml
[2024-09-26 18:19:16,058]-33-Reading yaml file param.yaml
[2024-09-26 18:19:16,061]-33-Reading yaml file schema.yaml
[2024-09-26 18:19:16,067]-46-Creating Directory
[2024-09-26 18:19:16,068]-46-Creating Directory
[2024-09-26 18:19:16,070]-10-Reading the Csv file artifacts/data_ingestion/WineQT.csv
True
